In [1]:
import os
import torch
from langchain.prompts.chat import (
    ChatPromptTemplate, 
    HumanMessagePromptTemplate, 
    SystemMessagePromptTemplate
)
from langchain import PromptTemplate
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (UnstructuredMarkdownLoader, 
                                        UnstructuredURLLoader)
from langchain.chains import (LLMChain, 
                              SimpleSequentialChain, 
                              retrieval_qa, 
                              conversational_retrieval
                              )
from transformers import (BitsAndBytesConfig, 
                          AutoModelForCausalLM, 
                          AutoTokenizer, 
                          GenerationConfig, 
                          pipeline
                          )
import warnings
warnings.filterwarnings('ignore')

os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 文本生成pipeline

In [2]:
MODEL_NAME = "../model_local/qwen/Qwen1.5-7B-Chat"
"""
quantiation_config = BitsAndBytesConfig(...)。这里， 使用BitsAndBytesConfig定义量化
配置。量化是一种降低深度学习模型的内存和计算需求的技术，
通常通过使用更少的比特（在外面的情况下为4bit）来表示模型参数
"""
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True
)
# 初始化大模型的tokenizer， 允许预处理输入到模型的文本数据
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, 
                                          use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
# 初始化用于因果建模的预训练语言模型， 
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, 
                                             torch_dtype=torch.float16, 
                                             trust_remote_code=True, 
                                             device_map="auto", 
                                             quantization_config=quantization_config)
# 为模型创建一个生成配置， 制定各种与生成相关的设置， 如令牌的最大数量、采样温度、top-p采样
# 与重复惩罚
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
# 使用pipeline函数创建一个文本生成管道， 这个管道是为文本生成而设置， 它将预训练的模型、
# 标记器和生成配置作为输入，它被配置为返回全文输出。
pipeline = pipeline("text-generation", 
                    model=model, 
                    tokenizer=tokenizer, 
                    generation_config = generation_config
                    )


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.21s/it]


HuggingFacePipeline是一个允许在本地运行Hugging Face模型的类， 它用于访问和利用Hugging Face Model Hub上托管的各种预训练的ML模型。
在使用HuggingFacePipeline时间， 建议安装xformer， 以实现高效的内存注意力。


In [3]:
llm = HuggingFacePipeline(
    pipeline = pipeline
)


In [4]:

query = "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
result = llm(
    query
)
print(result)

Explain the difference between ChatGPT and open source LLMs in a couple of lines.


In [5]:

query = "左下眼皮一直在跳动 "
result = llm(
    query
)
print(result)

左下眼皮一直在跳动 请问这是怎么回事 这种情况通常被称为眼睑抽搐或眼睑痉挛，它可能是由于疲劳、压力、焦虑、缺乏睡眠、过度用眼、眼部受伤或某些药物的副作用引起的。在大多数情况下，这种现象是暂时的，不需要特别治疗，但如果持续时间较长或者伴有其他症状（如视力模糊、眼睛疼痛等），建议咨询眼科医生以排除可能的眼部问题。保持良好的生活习惯和充足的休息也有助于缓解眼睑抽搐。


### 五、Embedding模型

在配置基础LLM之后， 继续配置embedding模型。每个文档都应该转换为embedding向量，以便使用用户的查询进行语义搜索。该查询也应该被embedding。为了实现这一点， 将利用阿里的GTE模型。
使用HuggingFaceEmbeddings类， 这是本地管道包装器， 用于与hugging face hub上托管的GTE模型进行交互。

In [8]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name = "../embedding_model",
    model_kwargs = {"device":"cuda"},
    encode_kwargs = {"normalize_embeddings": True}
)


SyntaxError: invalid character '，' (U+FF0C) (23696722.py, line 2)

#### Prompt Template
PromptTemplate通过结构化Prompt格式使模型按照用户期望的格式输出， 模版包括指令、few-shot
例子以及适合特定任务的特定上下文和问题

In [ ]:
template = """
|im_start|<>
Act as a Machine Learning engineer who is teaching high school students.
<>

{text} |im_start|
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

NameError: name 'PromptTemplate' is not defined

In [ ]:

query = "Explain what are Deep Neural Networks in 2-3 sentences"
result = llm(prompt.format(text=query))
print(result)


[INST] <>
Act as a Machine Learning engineer who is teaching high school students.
<>

Explain what are Deep Neural Networks in 2-3 sentences [/INST]
Deep Neural Networks (DNNs) are a type of machine learning algorithm that consist of multiple layers of interconnected nodes, allowing them to learn and extract complex patterns from large datasets. These networks use backpropagation to adjust their weights and improve performance on various tasks such as image or speech recognition.
